# Extrator de estatísticas da detecção
Agrupa os dados das iterações por mediana e média para posterior análise. Recebe como entrada os arquivos do caderno Detector.ipynb

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm
from time import time
from timer import timer

# Arquivos de entrada
tables = ['table1.csv','table2.csv','table3.csv']

Extração dos dados

In [2]:
tic = time()
metrics = ['Accuracy','Recall','F1','AUC']
resultado = {}
resultado['HAI'] = []
resultado['Files'] = []
resultado['Selector'] = []
resultado['N_Samples'] = []
resultado['Test_Size'] = []
resultado['Contamination'] = []
resultado['Classifier_1s'] = []
for metric in metrics:
    resultado["Classifier_2s_"+metric+"_Median"]=[]
    resultado["Classifier_2s_"+metric+"_Mean"]=[]
    resultado[metric+"_1s_Median"]=[]
    resultado[metric+"_1s_Mean"]=[]
    resultado[metric+"_2s_Median"]=[]
    resultado[metric+"_2s_Mean"]=[]
for a,b,c in product(metrics,["Abs","Rel"],["Median","Mean"]):
    resultado['_'.join([a,b,c])]=[]
for a in ["FP","FN","FPR","FNR"]:
    resultado[a+"_1s_Median"]=[]
    resultado[a+"_1s_Mean"]=[]
for a,b in product(["FP","FN","FPR","FNR"],metrics):
    resultado[a+"_2s_Median_"+b]=[]
    resultado[a+"_2s_Mean_"+b]=[]
for table in tables:
    df = pd.read_csv(table)
    aux = df.groupby(['HAI','Files','Selector','N_Samples','Test_Size','Contamination']).count().reset_index()
    params = []
    for i in range(aux.shape[0]):
        params.append((aux.iloc[i,0],
                       aux.iloc[i,1],
                       aux.iloc[i,2],
                       aux.iloc[i,3],
                       aux.iloc[i,4],
                       aux.iloc[i,5]))
    del aux

    for param in tqdm(params):
        version = param[0]
        files = param[1]
        fselector = param[2]
        n_samples = str(param[3])
        test_size = str(param[4])
        contamination = str(param[5])
        data = df.query("HAI == '"+version+
                        "' and Files == '"+files+
                        "' and Selector == '"+fselector+
                        "' and N_Samples == "+n_samples+
                        " and Test_Size == "+test_size+
                        " and Contamination == "+contamination)
        for s1clf in data['Classifier_1s'].unique():
            resultado['HAI'].append(version)
            resultado['Files'].append(files)
            resultado['Selector'].append(fselector)
            resultado['N_Samples'].append(n_samples)
            resultado['Test_Size'].append(test_size)
            resultado['Contamination'].append(contamination)
            resultado['Classifier_1s'].append(s1clf)
            dataux = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '1step_only'")
            
            median_fp_1s = pd.Series(dataux['FP']).median()
            resultado['FP_1s_Median'].append(median_fp_1s)
            median_fn_1s = pd.Series(dataux['FN']).median()
            resultado['FN_1s_Median'].append(median_fn_1s)

            resultado['FPR_1s_Median'].append(median_fp_1s/(int(test_size)-int(contamination)))
            resultado['FNR_1s_Median'].append(median_fn_1s/int(contamination))

            mean_fp_1s = pd.Series(dataux['FP']).mean()
            resultado['FP_1s_Mean'].append(mean_fp_1s)
            mean_fn_1s = pd.Series(dataux['FN']).mean()
            resultado['FN_1s_Mean'].append(mean_fn_1s)

            resultado['FPR_1s_Mean'].append(mean_fp_1s/(int(test_size)-int(contamination)))
            resultado['FNR_1s_Mean'].append(mean_fn_1s/int(contamination))
            for metric in metrics:
                aux = {}
                diff = 0
                last = "1step_only"
                last_fp = 9999999
                median_1s = dataux[metric].median()
                resultado[metric+"_1s_Median"].append(median_1s)
                mean_1s = dataux[metric].mean()
                resultado[metric+"_1s_Mean"].append(mean_1s)
                if median_1s == 0.0:
                    for s2clf in data['Classifier_2s'].unique():
                        median_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")[metric].median()
                        median_fp_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")['FP'].median()
                        if median_2s-median_1s > diff:
                            diff = median_2s-median_1s
                            last = s2clf
                            last_fp = median_fp_2s
                        elif median_2s-median_1s == diff and last_fp > median_fp_2s:
                            last = s2clf
                            last_fp = median_fp_2s
                    resultado[metric+'_Abs_Median'].append(diff)
                    resultado[metric+"_Rel_Median"].append(np.nan)
                else:
                    abs = 0
                    for s2clf in data['Classifier_2s'].unique():
                        median_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")[metric].median()
                        median_fp_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")['FP'].median()
                        temp = (median_2s-median_1s)*100/median_1s
                        if temp > diff:
                            diff = temp
                            abs = median_2s-median_1s
                            last = s2clf
                            last_fp = median_fp_2s
                        elif temp == diff and last_fp > median_fp_2s:
                            last = s2clf
                            last_fp = median_fp_2s
                    resultado[metric+'_Abs_Median'].append(abs)
                    resultado[metric+"_Rel_Median"].append(diff)
                median_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+last+"'")[metric].median()
                median_fp_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+last+"'")['FP'].median()
                median_fn_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+last+"'")['FN'].median()
                
                resultado['Classifier_2s_'+metric+'_Median'].append(last)
                resultado[metric+"_2s_Median"].append(median_2s)
                
                resultado['FP_2s_Median_'+metric].append(median_fp_2s)
                resultado['FN_2s_Median_'+metric].append(median_fn_2s)
                resultado['FPR_2s_Median_'+metric].append(median_fp_2s/(int(test_size)-int(contamination)))
                resultado['FNR_2s_Median_'+metric].append(median_fn_2s/int(contamination))
                
                diff = 0
                last = "1step_only"
                last_fp = 9999999
                if mean_1s == 0.0:
                    for s2clf in data['Classifier_2s'].unique():
                        mean_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")[metric].mean()
                        mean_fp_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")['FP'].mean()
                        if mean_2s-mean_1s > diff:
                            diff = mean_2s-mean_1s
                            last = s2clf
                            last_fp = mean_fp_2s
                        elif mean_2s-mean_1s == diff and last_fp > mean_fp_2s:
                            last = s2clf
                            last_fp = mean_fp_2s
                    resultado[metric+'_Abs_Mean'].append(diff)
                    resultado[metric+"_Rel_Mean"].append(np.nan)
                else:
                    abs = 0
                    for s2clf in data['Classifier_2s'].unique():
                        mean_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")[metric].mean()
                        mean_fp_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+s2clf+"'")['FP'].mean()
                        temp = (mean_2s-mean_1s)*100/mean_1s
                        if temp > diff:
                            diff = temp
                            abs = mean_2s-mean_1s
                            last = s2clf
                            last_fp = mean_fp_2s
                        elif temp == diff and last_fp > mean_fp_2s:
                            last = s2clf
                            last_fp = mean_fp_2s
                    resultado[metric+'_Abs_Mean'].append(abs)
                    resultado[metric+"_Rel_Mean"].append(diff)
                mean_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+last+"'")[metric].mean()
                mean_fp_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+last+"'")['FP'].mean()
                mean_fn_2s = data.query("Classifier_1s == '"+s1clf+"' and Classifier_2s == '"+last+"'")['FN'].mean()
                
                resultado['Classifier_2s_'+metric+'_Mean'].append(last)
                resultado[metric+"_2s_Mean"].append(mean_2s)
                
                resultado['FP_2s_Mean_'+metric].append(mean_fp_2s)
                resultado['FN_2s_Mean_'+metric].append(mean_fn_2s)
                resultado['FPR_2s_Mean_'+metric].append(mean_fp_2s/(int(test_size)-int(contamination)))
                resultado['FNR_2s_Mean_'+metric].append(mean_fn_2s/int(contamination))
toc = time()
timer(toc-tic)

100%|███████████████████████████████████████████████████████████████████████████████████| 306/306 [40:59<00:00,  8.04s/it]

Done in 1h 16m 36s


Salvando resultado em arquivo .csv

In [3]:
pd.DataFrame(resultado).to_csv("Stats.csv",index=False)